# Exercise 1 | Inspecting a Bitcoin address

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import json
from pprint import pprint

import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api

## Notebook Setup

This notebook assumes that you already followed the instructions outlined in `README.md` and installed the [GraphSense Python API](https://github.com/graphsense/graphsense-python) library locally.

First, we must setup our notebook and establish a connection to some hosted GraphSense instance.

We connect to a GraphSense instance hosted by [Iknaio](https://www.ikna.io/) and enter our API key in the provided `config.json` configuration file. An API key will be provided during the tutorial. If you would like to get an API key later, drop an email to contact@iknaio.com

## Load host and API key from config

In [10]:
f = open('config.json')
config = json.load(f)
f.close()

## Configure GraphSense client

Now we configure the GraphSense Python library

In [11]:
configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

We can test if our libary is working by retrieving summary statistics on supported ledgers.

In [12]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    pprint(api_response)

{'currencies': [{'name': 'btc',
                 'no_address_relations': 8007069316,
                 'no_addresses': 1118041725,
                 'no_blocks': 785717,
                 'no_entities': 506650685,
                 'no_labels': 11478,
                 'no_tagged_addresses': 269790236,
                 'no_txs': 824652501,
                 'timestamp': 1681686129},
                {'name': 'bch',
                 'no_address_relations': 2427116960,
                 'no_addresses': 336951901,
                 'no_blocks': 788823,
                 'no_entities': 151973521,
                 'no_labels': 143,
                 'no_tagged_addresses': 15017913,
                 'no_txs': 366604513,
                 'timestamp': 1681768466},
                {'name': 'ltc',
                 'no_address_relations': 1725595832,
                 'no_addresses': 182710535,
                 'no_blocks': 2457669,
                 'no_entities': 78384312,
                 'no_labels': 181,

# Address API

We start by retrieving some summary statistics on the Internet Archive's donation address. The response basically corresponds to the data shown in the property box of the GraphSense dashboard.

In [13]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = addresses_api.AddressesApi(api_client)

    currency = "btc"
    address = '1Archive1n2C579dMsAu3iC6tWzuQJz8dN'
    include_tags = True

    try:
        resp_addr = api_instance.get_address(currency, address)
        pprint(resp_addr)
    except graphsense.ApiException as e:
        print("Exception when calling AddressesApi->get_address: %s\n" % e)

{'address': '0x49bd56b275395130b41d67e1304b2d49f8a88725',
 'balance': {'fiat_values': [{'code': 'eur', 'value': 809.53},
                             {'code': 'usd', 'value': 891.7}],
             'value': 443050788154310941},
 'currency': 'eth',
 'entity': 219247462,
 'first_tx': {'height': 14680130,
              'timestamp': 1651248019,
              'tx_hash': 'a173b6c171138c8ec71f7f6badd931af8e62b9de8e9b7eaba3563b08877e436a'},
 'in_degree': 6,
 'is_contract': False,
 'last_tx': {'height': 16828820,
             'timestamp': 1678828811,
             'tx_hash': '6c50c28d3b9d2674c72ca52dcbd5796851e960e067f829dfd8010dadcf17b994'},
 'no_incoming_txs': 9,
 'no_outgoing_txs': 16,
 'out_degree': 8,
 'status': 'clean',
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 1474.33},
                                    {'code': 'usd', 'value': 1521.65}],
                    'value': 1162991822615963000},
 'total_spent': {'fiat_values': [{'code': 'eur', 'value': 1278.81},
             

In [14]:
print(f"Address {resp_addr.address} received {resp_addr.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_addr.in_degree} addresses.")

Address 0x49bd56b275395130b41d67e1304b2d49f8a88725 received 1474.33 EUR from 6 addresses.


# Entities API

Next, we inspect the corresponding entity, which clusters other addresses that are likely conrolled by the owner of that address. The entity ID is contained in the address response.

In [15]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = entities_api.EntitiesApi(api_client)

    currency = "btc" 
    entity = resp_addr.entity 
    include_tags = True 

    try:
        resp_entity = api_instance.get_entity(currency, entity)
        pprint(resp_entity)
    except graphsense.ApiException as e:
        print("Exception when calling EntitiesApi->get_entity: %s\n" % e)

{'balance': {'fiat_values': [{'code': 'eur', 'value': 0.0},
                             {'code': 'usd', 'value': 0.0}],
             'value': 0},
 'currency': 'btc',
 'entity': 219247462,
 'first_tx': {'height': 450740,
              'timestamp': 1485786538,
              'tx_hash': 'a9341814cdd188fb3e6844ba418fff3ccdf8fd372ef0f59e1cdc20df68b66e16'},
 'in_degree': 2,
 'last_tx': {'height': 459518,
             'timestamp': 1490825653,
             'tx_hash': 'e55b13defa09bdcc1b8d98d119f8bdc0355d1b4044070e0f2fdcb339fe2342f4'},
 'no_address_tags': 0,
 'no_addresses': 2,
 'no_incoming_txs': 2,
 'no_outgoing_txs': 1,
 'out_degree': 2,
 'root_address': '32rtRy5G5Sza43uwCMjtyc4Mv7U2ABDXPS',
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 376.62},
                                    {'code': 'usd', 'value': 405.91}],
                    'value': 38774921},
 'total_spent': {'fiat_values': [{'code': 'eur', 'value': 374.66},
                                 {'code': 'usd', 'value':

In [16]:
print(f"Entity {resp_entity.entity} likely controls {resp_entity.no_addresses} addresses, " +
      f"which received {resp_entity.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_entity.in_degree} entities.")

Entity 219247462 likely controls 2 addresses, which received 376.62 EUR from 2 entities.


# Entity Neighbors

Finally, we retrieve all neighbor entities receiving funds from our focus entity. Since it is more covenient to work with dataframes, we use the GraphSense Bulk API interface to retrieve a dataframe of receiving neighbor nodes.

In [17]:
import pandas as pd

data = {'entity': [resp_addr.entity]}

entityDF = pd.DataFrame.from_dict(data)

In [18]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = bulk_api.BulkApi(api_client)

    CURRENCY = "btc"
    operation = "list_entity_neighbors"
    body = {'entity': entityDF['entity'].to_list(),
            'direction': 'out',
            'include_labels': True}

    try:
        respDF = pd.read_csv(
                    api_instance.bulk_csv(CURRENCY, operation, body=body,
                                          num_pages=2,
                                          _preload_content=False))
    except graphsense.ApiException as e:
        print("Exception when calling BulkApi->bulk_csv: %s\n" % e)

In [19]:
respDF

,_error,_info,_request_entity,entity_actors,entity_balance_eur,entity_balance_usd,entity_balance_value,entity_best_address_tag_abuse,entity_best_address_tag_actor,entity_best_address_tag_address,...,entity_total_spent_value,entity_total_tokens_received,entity_total_tokens_spent,labels,labels_count,no_txs,token_values,value_eur,value_usd,value_value
0,NaN,NaN,219247462,NaN,0.0,0.0,0,NaN,NaN,NaN,...,108000000,NaN,NaN,NaN,0,1,NaN,193.25,208.08,20000000
1,NaN,NaN,219247462,NaN,0.0,0.0,0,NaN,NaN,NaN,...,18685630,NaN,NaN,NaN,0,1,NaN,180.55,194.40,18685630


We filter the result and see the relations between known entity IDs. We also see two known receiving entities.

In [20]:
resultDF = respDF[['_request_entity', 'entity_entity', 'labels', 'no_txs']] \
    .rename(columns={'_request_entity': 'src', 'entity_entity': 'dst', 'labels': 'dst_label'}) \
    .dropna()
resultDF

,src,dst,dst_label,no_txs
